In [1]:
import pandas as pd
import boto3
import json

# Load DWH Params from a file

In [2]:
import configparser
config = configparser.ConfigParser()
config.read_file(open('dwh.cfg'))

KEY                    = config.get('AWS','KEY')
SECRET                 = config.get('AWS','SECRET')

DWH_CLUSTER_TYPE       = config.get("CLUSTER","TYPE")
DWH_NUM_NODES          = config.get("CLUSTER","NUM_NODES")
DWH_NODE_TYPE          = config.get("CLUSTER","NODE_TYPE")

DWH_CLUSTER_IDENTIFIER = config.get("CLUSTER","IDENTIFIER")
DWH_DB                 = config.get("CLUSTER","DB_NAME")
DWH_DB_USER            = config.get("CLUSTER","DB_USER")
DWH_DB_PASSWORD        = config.get("CLUSTER","DB_PASSWORD")
DWH_PORT               = config.get("CLUSTER","DB_PORT")

DWH_IAM_ROLE_NAME      = config.get("IAM_ROLE", "NAME")

### Create clients

In [3]:
session = boto3.session.Session(aws_access_key_id=KEY, aws_secret_access_key=SECRET)
iam = session.client('iam', 'us-west-2')
redshift = session.client('redshift', 'us-west-2')

### Create IAM role

In [ ]:
try:
    dwhRole = iam.create_role(
        Path='/',
        RoleName=DWH_IAM_ROLE_NAME,
        AssumeRolePolicyDocument=json.dumps(
            dict(
                Statement=[
                    dict(
                        Action='sts:AssumeRole',
                        Effect='Allow',
                        Principal=dict(
                            Service='redshift.amazonaws.com'
                        )
                    )
                ],
                Version='2012-10-17'
            )
        ),
        Description='Allow Redshift clusters to call AWS services on my behalf',
    )
    

except Exception as e:
    print(e)

### Attach policy to IAM role

In [ ]:
try:
    iam.attach_role_policy(
        RoleName=DWH_IAM_ROLE_NAME,
        PolicyArn='arn:aws:iam::aws:policy/AmazonS3ReadOnlyAccess'
    )
except Exception as e:
    print(e)

### Fetch role ARN

In [ ]:
roleArn = iam.get_role(
    RoleName=DWH_IAM_ROLE_NAME
)['Role']['Arn']

### Create Redshift cluster

In [ ]:
try:
    response = redshift.create_cluster(        
        ClusterType=DWH_CLUSTER_TYPE,
        NodeType=DWH_NODE_TYPE,
        NumberOfNodes=int(DWH_NUM_NODES),
        
        DBName=DWH_DB,
        ClusterIdentifier=DWH_CLUSTER_IDENTIFIER,
        MasterUsername=DWH_DB_USER,
        MasterUserPassword=DWH_DB_PASSWORD,
        
        IamRoles=[roleArn]
    )
except Exception as e:
    print(e)

### Get cluster status until available

In [ ]:
import time

def prettyRedshiftProps(props):
    pd.set_option('display.max_colwidth', -1)
    keysToShow = ["ClusterIdentifier", "NodeType", "ClusterStatus", "MasterUsername", "DBName", "Endpoint", "NumberOfNodes", 'VpcId']
    x = [(k, v) for k,v in props.items() if k in keysToShow]
    return pd.DataFrame(data=x, columns=["Key", "Value"])

def fetch_props():
    res = redshift.describe_clusters(ClusterIdentifier=DWH_CLUSTER_IDENTIFIER)['Clusters'][0]
#     time.sleep(10)
    return res

In [ ]:
myClusterProps = fetch_props()
res = prettyRedshiftProps(myClusterProps)

# while res['Value'][res['Key']=='ClusterStatus'].values[0] != 'available':
#     res = prettyRedshiftProps(fetch_props())

In [4]:
from helpers import fetch_props, prettyRedshiftProps

status, host = prettyRedshiftProps(fetch_props(client=redshift, cluster_id=DWH_CLUSTER_IDENTIFIER))

In [6]:
status

'deleted'

In [ ]:
res['Value'][res['Key']=='Endpoint'].values[0]['Address']

### Get cluster endpo

In [ ]:
HOST = myClusterProps['Endpoint']['Address']

### Clean up resources

<b><font color='red'>DO NOT RUN THIS UNLESS YOU ARE SURE</b>

In [ ]:
def clean_up(cluster_id, iam_role_name):
    redshift.delete_cluster( ClusterIdentifier=DWH_CLUSTER_IDENTIFIER,  SkipFinalClusterSnapshot=True)
    
    myClusterProps = fetch_props()
    res = prettyRedshiftProps(myClusterProps)

    while res['Value'][res['Key']=='ClusterStatus'].values[0] != 'deleted':
        res = prettyRedshiftProps(fetch_props())

    iam.detach_role_policy(RoleName=DWH_IAM_ROLE_NAME, PolicyArn="arn:aws:iam::aws:policy/AmazonS3ReadOnlyAccess")
    iam.delete_role(RoleName=DWH_IAM_ROLE_NAME)
    
    return

In [ ]:
clean_up()

In [ ]:
myClusterProps = fetch_props()
res = prettyRedshiftProps(myClusterProps)

while res['Value'][res['Key']=='ClusterStatus'].values[0] != 'deleted':
    res = prettyRedshiftProps(fetch_props())